# Practica cuaderno #6

## Practica.

Realizar la planificación y modelado de una simulacion basado en eventos discretos para identificar los contagios dentro de un establecimiento de estudiantes post COVID-19 para el reingreso de clases. Los pasos para el regreso progresivo a clases se describe a continuación:

            ▪ Calcular e Identificar el aforo máximo permitido en la institución educativa.
            ▪ Identificar el número de estudiantes y el porcentaje de contagio dentro del contexto poblacional el mismo que se aplicara dentro de la simulación por el contagio.
            ▪ Generar la simulación desde la puerta de ingreso y la salida del plantel por día.
            ▪ Los estudiantes hacen fila para ingresar al establecimiento y no pueden superar el aforo permitido.
            ▪ Realizar con un tiempo de un 1 mes.

Generar graficas que indiquen los estudiantes que presentaron complicaciones y el total de estudiantes por establecimiento en cada día o semana.

**Nota**: No tomar datos de transporte, numero de aulas, personal docente, etc. Solo se el ingreso, salida y el contagio dentro del establecimiento.


In [26]:
!pip install faker

Defaulting to user installation because normal site-packages is not writeable


In [27]:
import simpy
from scipy.stats import bernoulli
import random
from faker import Faker

### Constantes de interes

Se establecen constantes acorde a lo establecido en el enunciado anterior, además de esto se identifica valores obtenidos del mundo real como el tiempo empleado dentro de la institucion o el aforo permitido en un establecimiento.


In [28]:
# AFORO NORMAL EN LA UNIVERSIDAD 'ASUS'
AFORO_NORMAL = 1_200
# AFORO MAXIMO PERMITIDO (70% de la capacidad maxima)
AFORO_POST_COVID = int(AFORO_NORMAL * 0.7)
# TIEMPO DE SIMULACION 1 mes por 24 horas por 60 minutos restado en promedio 4.5 fines de semana por 2 dias por 24 horas
TIEMPO_SIMULACION = (30 * 24 * 60) - (4.5 * 2 * 24 * 60)
# TIEMPO DE ESTUDIO 8 horas por 60 minutos
TIEMPO_ESTUDIO = (8 * 60)

# PROBABILIDAD DE CONTAGIO EN BASE A LOS DATOS DE LA OMS IGNORANDO MEDIDAS PREVENTIVAS
PROBABILIDAD_CONTAGIO = 0.632

In [29]:
contagios_totales = []
fake = Faker()

### Proceso de simulacion

Se define las clases, metodos y funciones que permiten la simulacion de contagios en un establecimiento.

In [30]:
def calcular_estado_contagio():
    """
    Simula el contagio de un estudiante en base a la probabilidad de contagio (63.2%).
    El resultado de contagio es almacenado dentro de una lista que luego sera utilizada para graficar los contagios por una unidad de tiempo determinada.
    :return: None
    """

    global PROBABILIDAD_CONTAGIO, contagios_totales
    contagios_totales.append(bernoulli.rvs(PROBABILIDAD_CONTAGIO))


class Universidad:
    def __init__(self, environment: simpy.Environment):
        """
        Constructor para la clase Universidad.
        Inicializa las variables y crea un recurso compartido dentro del ambiente de simulacion
        para simular un contenedor de estudiantes.

        :param environment: Ambiente en el cual se desarrolla la simulacion.
        """
        self.environment = environment
        self.aforo: simpy.Resource = simpy.Resource(environment, AFORO_POST_COVID)

    def estudiar(self):
        """
        Simula el estudio y contagio del estudiante en base a la distribucion Bernoulli antes establecida.
        Además crea un delay en la ejecucion simulando el tiempo de estudio.
        :return: None
        """
        calcular_estado_contagio()
        yield self.environment.timeout(TIEMPO_ESTUDIO)

    def arrivar_estudiante(self, nombre):
        # print(f'El estudiante: {nombre} llega a la institucion al minuto: {self.environment.now}')
        with self.aforo.request() as a:
            yield a
            yield self.environment.process(self.estudiar())
            # print(f'El estudiante: {nombre} sale al minuto: {self.environment.now}')

def simular_contagios(environment: simpy.Environment):
    """
    Simula el proceso de estudio y contagio de un estudiante por un tiempo indefinido.

    :param environment: Ambiente en el cual se realiza la simulacion.
    :return: Retorna un delay de 1 minuto.
    """
    universidad = Universidad(environment)
    while True:
        yield environment.timeout(1)
        environment.process(universidad.arrivar_estudiante(fake.name()))

### Ejecucion de la simulacion

In [31]:
random.seed(42)
environment = simpy.Environment()
environment.process(simular_contagios(environment))
environment.run(until=TIEMPO_SIMULACION)

### Procesamiento de los datos para graficar

In [32]:
check_condition = lambda index : index % (((30 * 24 * 60) - (4.5 * 2 * 24 * 60))/30) != 0 or index == 0
update_dictionary = lambda dictionary, key: (dictionary.update({key: dictionary.get(key)+1}) )

contagios_por_dia: dict[str, int] = {f'DIA {x+1}': 0 for x in range(30)}
sanos_por_dia: dict[str, int] = {f'DIA {x+1}': 0 for x in range(30)}

dia_actual = 1
for index, value in enumerate(contagios_totales):
    if check_condition(index):
        if value == 0:
            update_dictionary(sanos_por_dia, f'DIA {dia_actual}')
        else:
            update_dictionary(contagios_por_dia, f'DIA {dia_actual}')
    else:
        dia_actual += 1

### Graficas de los resultados de contagios vs sanos.

In [33]:
import plotly.graph_objects as go
animals=list(contagios_por_dia.keys())

fig = go.Figure(data=[
    go.Bar(name='Sanos', x=animals, y=list(sanos_por_dia.values())),
    go.Bar(name='Contagios', x=animals, y=list(contagios_por_dia.values()))
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()